# pyABC Tutorial

This is a tutorial for the python package **pyABC** for **likelihood-free parameter inference using approximate Bayesian computation**. See:
* https://github.com/ICB-DCM/pyABC for the code
* https://pyabc.readthedocs.io/en/latest/ for the documentation and examples
* https://pyabc.readthedocs.io/en/latest/cite.html for the papers
* https://github.com/yannikschaelte/pres_pyabc_inverse_2022 for presentation slides and further material

## Install

### Python

This step can be skipped if you already have python>=3.8 (check via `python3 --version`).
Download the conda package manager via the mini distribution [Miniconda](https://docs.conda.io/en/latest/miniconda.html), see the [instructions](https://docs.conda.io/projects/conda/en/latest/user-guide/install/index.html).
Specifically, for Linux, execute

```sh
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
bash Miniconda3-latest-Linux-x86_64.sh -b -p miniconda3
rm Miniconda3-latest-Linux-x86_64.sh
```

### Custom environment

We will work in a fresh local environment:

```sh
eval "$(miniconda3/bin/conda shell.bash hook)"
conda create -y -n invenv python
```

### (Re-)Activate environment

To activate the local environment, in each new shell session execute:

```sh
eval "$(miniconda3/bin/conda shell.bash hook)"
conda activate invenv
```

To verify your installation: `which pip` should now point to your newly created environment.

### Install dependencies

We install pyABC, as well as the interactive jupyter platform:

```sh
pip install jupyterlab pyabc
```

To open a jupyter notebook (e.g. this one):

```sh
jupyter lab [DIR]
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyabc

pyabc.settings.set_figure_params('pyabc')  # for beautified plots

## Toy example

As toy example, we consider a model $y\sim\mathcal{N}(\mu,\sigma^2)$ with fixed variance $\sigma^2=0.5^2$ and unknown univariate mean $\mu$. We have observed data $y_\text{obs} = 3.5$, and assume prior knowledge that the true man is in the interval $\mu \in [1,10]$.

### Own ABC

To demonstrate how simple ABC is at its core, let us first implement an own rejection algorithm with quadratic distance, acceptance threshold $\varepsilon = 0.1$, and population size $N=1000$.
Define model, distance, and prior:

In [ ]:
from scipy.stats import uniform, norm

# observed data
y_obs = 3.5

# noise standard deviation
sigma = 0.5

# model
# TODO def model(p):

# distance
# TODO def distance(y, y_obs):

# prior
xmin = 1
xmax = 7
# TODO prior = ...

Implement the ABC algorithm and plot the posterior:

In [ ]:
# population size 
N = 1000

# acceptance threshold
eps = 0.1

# accepted parameters
thetas = []

# ABC loop: sample, simulate, evaluate until enough acceptances
# TODO

# plot
fig, ax = plt.subplots()
ax.hist(thetas, bins=int(N / 50), density=True)
ax.set_xlim(xmin, xmax)
ax.set_xlabel(r"$\mu$");

We can compare this to the true posterior:

In [ ]:
from scipy.integrate import quad

# TODO

### Own ABC-IS

As the prior may be uninformative, in importance sampling we use a different proposal distribution to sample from (and afterwards reweight the particles by prior divided by proposal):

In [ ]:
from scipy.stats import norm

# proposal distribution
proposal = norm(loc=7, scale=2)

# accepted parameters and corresponding weights
thetas = []
weights = []

# ABC loop: sample, simulate, evaluate until enough acceptances
# TODO

# plot
fig, ax = plt.subplots()
ax.hist(thetas, weights=weights, bins=int(N / 50), density=True)
ax.set_xlim(xmin, xmax)
ax.set_xlabel(r"$\mu$")

xs = np.linspace(xmin, xmax, 100)
ax.plot(xs, posterior(xs), color="grey", linestyle="dashed");

What is our effective sample size? It is given as:

$$\operatorname{ESS} = \frac{\left(\sum_iw_i\right)^2}{\sum_i w_i^2}$$

In [ ]:
# TODO

### In pyABC

Now, let's use pyABC to perform the same analysis with its implemented ABC-SMC algorithm.

#### Problem description

Specify the model:

In [ ]:
sigma = 0.5

# pyABC accepts arbitrary model functions that return a dictionary of
#  observed values

# more precisely, the following structure is assumed:
#  model(p: Parameter) -> Simulation
#  where Parameter = dict[str, float]
#  and Simulation = dict[str, {float, np.ndarray, pd.DataFrame}]

# TODO def model(p):

The observed data are:

In [ ]:
# TODO y_obs = ...

The prior is specified via [`Distribution` and `RV`](https://pyabc.readthedocs.io/en/latest/api_random_variables.html#random-variables):

In [ ]:
# TODO prior = ...

Distance function:

In [ ]:
# distances are assumed to follow the structure
#  distance(y: Simulation, y_obs: Simulation) -> float

def distance(y, y_obs):
    return (y["y"] - y_obs["y"])**2

#### ABC analysis

The problem being defined, we continue by setting up an [`ABCSMC`](https://pyabc.readthedocs.io/en/latest/api_inference.html#inference) analysis with $N=1000$ samples per generation.
We have to specify where to log the analysis results.
Then, we're all good and can run an analysis that continues until 10 generations have been created, or the acceptance rate falls below $0.1$:

In [ ]:
# population size
N = 1000

# ABCSMC instance
# TODO abc = pyabc.ABCSMC(...)

# local sqlite database
db_path = "test.db"
abc.new("sqlite:///" + db_path, y_obs)

# TODO history = abc.run(...)

### Visualization and analysis

Let us [visualize](https://pyabc.readthedocs.io/en/latest/api_visualization.html) the ABC posterior approximation over the generations:

In [ ]:
# TODO pyabc.visualization.plot_kde_1d_highlevel(...)

Next, let us analyze the performance of the sampler by plotting sample numbers, epsilon threshold, and effective sample sizes over the generations:

In [ ]:
# TODO

That's it, you have run your first pyABC analysis. For more details, illustrations of the various algorithms pyABC implements, and application examples, see https://pyabc.readthedocs.io/en/latest/examples.html.

























































































































